In [1]:
import time
import pandas as pd
import numpy as np
from finstore.finstore import Finstore
from finstore.finstore_s3 import PartitionType

In [2]:
f = Finstore(market_name="crypto_binance", timeframe="1m", partition_type = PartitionType.DAILY)

Len items in 'database/finstore/market_name=crypto_binance/timeframe=1m': 78


# Generate Mock Data

In [3]:
num_days = 10
total_rows = num_days * 1440
all_dates = pd.date_range("2025-01-01", periods=total_rows, freq='min', tz='UTC')
print(f"Generated {len(all_dates)} timestamps.")

# Create random OHLCV data with a timestamp column
df = pd.DataFrame({
     'timestamp': all_dates,
     'open': np.random.rand(len(all_dates)),
     'high': np.random.rand(len(all_dates)),
     'low': np.random.rand(len(all_dates)),
     'close': np.random.rand(len(all_dates)),
     'volume': np.random.randint(100, 1000, size=len(all_dates))
})

Generated 14400 timestamps.


In [4]:
df

,timestamp,open,high,low,close,volume
0,2025-01-01 00:00:00+00:00,0.777260,0.566313,0.871219,0.823743,875
1,2025-01-01 00:01:00+00:00,0.864125,0.073993,0.060385,0.531568,900
2,2025-01-01 00:02:00+00:00,0.674777,0.113464,0.678046,0.038857,278
3,2025-01-01 00:03:00+00:00,0.425262,0.802596,0.452586,0.242788,727
4,2025-01-01 00:04:00+00:00,0.156033,0.400442,0.636652,0.090131,523
...,...,...,...,...,...,...
14395,2025-01-10 23:55:00+00:00,0.894074,0.769238,0.574076,0.727304,634
14396,2025-01-10 23:56:00+00:00,0.787833,0.271091,0.678884,0.921527,506
14397,2025-01-10 23:57:00+00:00,0.189437,0.191940,0.131817,0.163295,953
14398,2025-01-10 23:58:00+00:00,0.107091,0.736420,0.807895,0.122124,138


# Store Data with Daily Partitioning

In [5]:
%%time
f.s3.store(symbol="BTC/USDT", df=df)

Stored 1440 rows to s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-01/ohlcv_data.parquet
Stored 1440 rows to s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-02/ohlcv_data.parquet
Stored 1440 rows to s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-03/ohlcv_data.parquet
Stored 1440 rows to s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-04/ohlcv_data.parquet
Stored 1440 rows to s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-05/ohlcv_data.parquet
Stored 1440 rows to s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-06/ohlcv_data.parquet
Stored 1440 rows to s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-07/ohlcv_data.parquet
Stored 1440 rows to s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-08/ohlcv_data.parquet
Stored 1440 rows to s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-09/ohlcv_data.parquet
Stored 1440 rows to s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-10/ohlcv_data.parquet
CPU times: user 275 

# Fetching Data

In [6]:
%%time
fetched_df = f.s3.fetch(symbol="BTC/USDT", 
                        start_date="2025-01-01", 
                        end_date="2025-01-10")

CPU times: user 358 ms, sys: 86.6 ms, total: 445 ms
Wall time: 251 ms


In [8]:
%%time
fetched_df = f.s3.fetch(symbol="BTC/USDT", 
                        start_date="2025-01-01", 
                        end_date="2025-01-5")

CPU times: user 74.3 ms, sys: 24 ms, total: 98.3 ms
Wall time: 86.3 ms


# Deleting Data

In [9]:
%%time
f.s3.delete(symbol="BTC/USDT", partition_value="2025-01-05")

Deleted s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-05/ohlcv_data.parquet
CPU times: user 8.97 ms, sys: 0 ns, total: 8.97 ms
Wall time: 29.3 ms


In [10]:
f.s3.fetch(symbol="BTC/USDT", 
            start_date="2025-01-01", 
            end_date="2025-01-10")

,timestamp,open,high,low,close,volume,partition
0,2025-01-01 00:00:00+00:00,0.777260,0.566313,0.871219,0.823743,875,2025-01-01
1,2025-01-01 00:01:00+00:00,0.864125,0.073993,0.060385,0.531568,900,2025-01-01
2,2025-01-01 00:02:00+00:00,0.674777,0.113464,0.678046,0.038857,278,2025-01-01
3,2025-01-01 00:03:00+00:00,0.425262,0.802596,0.452586,0.242788,727,2025-01-01
4,2025-01-01 00:04:00+00:00,0.156033,0.400442,0.636652,0.090131,523,2025-01-01
...,...,...,...,...,...,...,...
12955,2025-01-10 23:55:00+00:00,0.894074,0.769238,0.574076,0.727304,634,2025-01-10
12956,2025-01-10 23:56:00+00:00,0.787833,0.271091,0.678884,0.921527,506,2025-01-10
12957,2025-01-10 23:57:00+00:00,0.189437,0.191940,0.131817,0.163295,953,2025-01-10
12958,2025-01-10 23:58:00+00:00,0.107091,0.736420,0.807895,0.122124,138,2025-01-10


In [11]:
%%time
f.s3.delete(symbol="BTC/USDT")

Deleted s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-01/ohlcv_data.parquet
Deleted s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-02/ohlcv_data.parquet
Deleted s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-03/ohlcv_data.parquet
Deleted s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-04/ohlcv_data.parquet
Deleted s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-06/ohlcv_data.parquet
Deleted s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-07/ohlcv_data.parquet
Deleted s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-08/ohlcv_data.parquet
Deleted s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-09/ohlcv_data.parquet
Deleted s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-10/ohlcv_data.parquet
CPU times: user 38.4 ms, sys: 0 ns, total: 38.4 ms
Wall time: 93.2 ms
